In [1]:
!nvidia-smi

Fri Dec 21 23:26:25 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.145                Driver Version: 384.145                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   33C    P8    17W / 250W |     10MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 23%   34C    P8    17W / 250W |     10MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import logging
import math
import numpy as np
logging.getLogger().setLevel(logging.INFO)

from tqdm import tqdm
from idst_util import trivial
from idst_util import dstc2

# Make sure data is available
trivial.print_idst()
dstc2.check()

# Retrieve raw data
raw_X_train, raw_Y_train, \
raw_X_dev, raw_Y_dev, \
raw_X_test, raw_X_test, \
ontology = dstc2.retrieve_raw_datasets(train_data_augmentation = True)

INFO:root:+--------------------------------+
INFO:root:|         _ ____  ___________    |
INFO:root:|        (_) __ \/ ___/_  __/    |
INFO:root:|       / / / / /\__ \ / /       |
INFO:root:|      / / /_/ /___/ // /        |
INFO:root:|     /_/_____//____//_/         |
INFO:root:|                                |
INFO:root:+--------------------------------+
INFO:root:|Incremental Dialog State Tracker|
INFO:root:+--------------------------------+
INFO:root:+--------------------------------+
INFO:root:|     Dialog State Tracker 2     |
INFO:root:|         Data Checker           |
INFO:root:+--------------------------------+
INFO:root:Looking for dstc2 directory in .
INFO:root:dstc2 was found!
INFO:root:Looking for dstc2_traindev directory in ./dstc2
INFO:root:dstc2_traindev was found!
INFO:root:Looking for dstc2_test directory in ./dstc2
INFO:root:dstc2_test was found!
INFO:root:Looking for dstc2_scripts directory in ./dstc2
INFO:root:dstc2_scripts was found!
INFO:root:Done!
INFO:root:+-

In [3]:
logging.info("+--------------------------------+")
logging.info("|            Baseline            |")
logging.info("+--------------------------------+")

GPU_ID = 0
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
#DEVICE = "cpu"
if DEVICE == "cpu":
    logging.warning("Running on CPU")
else:
    logging.info("Running on GPU")

INFO:root:+--------------------------------+
INFO:root:|            Baseline            |
INFO:root:+--------------------------------+
INFO:root:Running on GPU


In [4]:
logging.info("+--------------------------------+")
logging.info("|          Vocabulary            |")
logging.info("+--------------------------------+")
logging.info("Creating token_to_index, index_to_token and token_to_count dictionaries")

token_to_index = {"<unk>": 0}
index_to_token = {0: "<unk>"}
token_to_count = {"<unk>": 1}
max_sequence_length = 0

for raw_train_dialog in tqdm(raw_X_train):
    for raw_train_turn in raw_train_dialog:
        
        current_sequence_length = len(raw_train_turn["system"]) + len(raw_train_turn["user"])
        if current_sequence_length > max_sequence_length:
            max_sequence_length = current_sequence_length
        
        for system_token in raw_train_turn["system"]:
            token = system_token[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
        
        for user_token in raw_train_turn["user"]:
            token = user_token[0]
            if token not in token_to_index:
                token_to_index[token] = len(token_to_index)
                index_to_token[len(token_to_index)] = token
                token_to_count[token] = 1
            else:
                token_to_count[token] += 1
                
assert len(token_to_index) == len(index_to_token)
assert len(token_to_index) == len(token_to_count)

logging.info("Vocabulary length:\t{}".format(len(token_to_index)))
logging.info("Max sequence length:\t{}".format(max_sequence_length))

INFO:root:+--------------------------------+
INFO:root:|          Vocabulary            |
INFO:root:+--------------------------------+
INFO:root:Creating token_to_index, index_to_token and token_to_count dictionaries
100%|██████████| 3224/3224 [00:00<00:00, 35676.71it/s]
INFO:root:Vocabulary length:	1156
INFO:root:Max sequence length:	28


In [5]:
logging.info("+--------------------------------+")
logging.info("|         Configuration          |")
logging.info("+--------------------------------+")

VOCABULARY_SIZE = len(token_to_index)
EMBEDDING_DIM = 170
HIDDEN_DIM = 100
NUM_NODES = 300

METHOD_DIM = len(ontology["method"])
REQUESTED_DIM = int(math.pow(2, len(ontology["requestable"])))
GOAL_FOOD_DIM = len(ontology["informable"]["food"]) + 2
GOAL_PRICERANGE_DIM = len(ontology["informable"]["pricerange"]) + 2
GOAL_NAME_DIM = len(ontology["informable"]["name"]) + 2
GOAL_AREA_DIM = len(ontology["informable"]["area"]) + 2

logging.info("VOCABULARY_SIZE:\t{}".format(VOCABULARY_SIZE))
logging.info("EMBEDDING_DIM:\t{}".format(EMBEDDING_DIM))
logging.info("HIDDEN_DIM:\t\t{}".format(HIDDEN_DIM))
logging.info("NUM_NODES:\t\t{}".format(NUM_NODES))
logging.info("METHOD_DIM:\t\t{}".format(METHOD_DIM))
logging.info("REQUESTED_DIM:\t{}".format(REQUESTED_DIM))
logging.info("GOAL_FOOD_DIM:\t{}".format(GOAL_FOOD_DIM))
logging.info("GOAL_PRICERANGE_DIM:\t{}".format(GOAL_PRICERANGE_DIM))
logging.info("GOAL_NAME_DIM:\t{}".format(GOAL_NAME_DIM))
logging.info("GOAL_AREA_DIM:\t{}".format(GOAL_AREA_DIM))

INFO:root:+--------------------------------+
INFO:root:|         Configuration          |
INFO:root:+--------------------------------+
INFO:root:VOCABULARY_SIZE:	1156
INFO:root:EMBEDDING_DIM:	170
INFO:root:HIDDEN_DIM:		100
INFO:root:NUM_NODES:		300
INFO:root:METHOD_DIM:		5
INFO:root:REQUESTED_DIM:	256
INFO:root:GOAL_FOOD_DIM:	93
INFO:root:GOAL_PRICERANGE_DIM:	5
INFO:root:GOAL_NAME_DIM:	115
INFO:root:GOAL_AREA_DIM:	7


In [6]:
class LecTrackEncoder(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocabulary_size, num_nodes, device):
        super(LecTrackEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.device = device
        self.token_embeddings = nn.Embedding(num_embeddings = vocabulary_size,
                                            embedding_dim = embedding_dim)
        self.linear = nn.Linear(in_features = (embedding_dim + 1),
                                out_features = num_nodes)
        self.lstm = nn.LSTM(num_nodes, hidden_dim)
        
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim, device = self.device),
                torch.zeros(1, 1, self.hidden_dim, device = self.device))
        
    def forward(self, indexes, scores):
        embeddings = self.token_embeddings(indexes)
        embeddings_concat_score = torch.cat((embeddings, scores.unsqueeze(dim = 1)), dim = 1) 
        altered_embeddings = F.relu(self.linear(embeddings_concat_score))
        lstm_out, self.hidden = self.lstm(altered_embeddings.view(len(indexes), 1, -1), self.hidden)
        return self.hidden
    
model_LecTrackEncoder = LecTrackEncoder(embedding_dim = EMBEDDING_DIM,
                                        hidden_dim = HIDDEN_DIM,
                                        vocabulary_size = VOCABULARY_SIZE,
                                        num_nodes = NUM_NODES,
                                        device = DEVICE)

model_LecTrackEncoder = model_LecTrackEncoder.to(DEVICE)

optimizer_LecTrackEncoder = optim.Adam(model_LecTrackEncoder.parameters(), lr = 1e-4)

In [7]:
class LecTrackMethodClassifier(nn.Module):
    
    def __init__(self, hidden_dim, method_dim):
        super(LecTrackMethodClassifier, self).__init__()
        self.method_dim = method_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = method_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.method_dim), dim = 1)

    
model_LecTrackMethodClassifier = LecTrackMethodClassifier(hidden_dim = HIDDEN_DIM,
                                                          method_dim = METHOD_DIM)

model_LecTrackMethodClassifier = model_LecTrackMethodClassifier.to(DEVICE)

optimizer_LecTrackMethodClassifier = optim.Adam(model_LecTrackMethodClassifier.parameters(), lr = 1e-4)

In [8]:
class LecTrackRequestedClassifier(nn.Module):
    
    def __init__(self, hidden_dim, requested_dim):
        super(LecTrackRequestedClassifier, self).__init__()
        self.requested_dim = requested_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = requested_dim) 
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.requested_dim), dim = 1)

model_LecTrackRequestedClassifier = LecTrackRequestedClassifier(hidden_dim = HIDDEN_DIM,
                                                                requested_dim = REQUESTED_DIM)

model_LecTrackRequestedClassifier = model_LecTrackRequestedClassifier.to(DEVICE)

optimizer_LecTrackRequestedClassifier = optim.Adam(model_LecTrackRequestedClassifier.parameters(), lr = 1e-4)

In [9]:
class LecTrackGoalFoodClassifier(nn.Module):
    
    def __init__(self, hidden_dim, goal_food_dim):
        super(LecTrackGoalFoodClassifier, self).__init__()
        self.goal_food_dim = goal_food_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = goal_food_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.goal_food_dim), dim = 1)

model_LecTrackGoalFoodClassifier = LecTrackGoalFoodClassifier(hidden_dim = HIDDEN_DIM,
                                                              goal_food_dim = GOAL_FOOD_DIM)

model_LecTrackGoalFoodClassifier = model_LecTrackGoalFoodClassifier.to(DEVICE)

optimizer_LecTrackGoalFoodClassifier = optim.Adam(model_LecTrackGoalFoodClassifier.parameters(), lr = 1e-4)

In [10]:
class LecTrackGoalPricerangeClassifier(nn.Module):
    
    def __init__(self, hidden_dim, goal_pricerange_dim):
        super(LecTrackGoalPricerangeClassifier, self).__init__()
        self.goal_pricerange_dim = goal_pricerange_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = goal_pricerange_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.goal_pricerange_dim), dim = 1)

model_LecTrackGoalPricerangeClassifier = LecTrackGoalPricerangeClassifier(hidden_dim = HIDDEN_DIM,
                                                                          goal_pricerange_dim = GOAL_PRICERANGE_DIM)

model_LecTrackGoalPricerangeClassifier = model_LecTrackGoalPricerangeClassifier.to(DEVICE)

optimizer_LecTrackGoalPricerangeClassifier = optim.Adam(model_LecTrackGoalPricerangeClassifier.parameters(), lr = 1e-4)

In [11]:
class LecTrackGoalNameClassifier(nn.Module):
    
    def __init__(self, hidden_dim, goal_name_dim):
        super(LecTrackGoalNameClassifier, self).__init__()
        self.goal_name_dim = goal_name_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = goal_name_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.goal_name_dim), dim = 1)
    
model_LecTrackGoalNameClassifier = LecTrackGoalNameClassifier(hidden_dim = HIDDEN_DIM,
                                                             goal_name_dim = GOAL_NAME_DIM)

model_LecTrackGoalNameClassifier = model_LecTrackGoalNameClassifier.to(DEVICE)

optimizer_LecTrackGoalNameClassifier = optim.Adam(model_LecTrackGoalNameClassifier.parameters(), lr = 1e-4)

In [12]:
class LecTrackGoalAreaClassifier(nn.Module):
    
    def __init__(self, hidden_dim, goal_area_dim):
        super(LecTrackGoalAreaClassifier, self).__init__()
        self.goal_area_dim = goal_area_dim
        self.linear = nn.Linear(in_features = hidden_dim,
                                out_features = goal_area_dim)
        
    def forward(self, hidden):
        return F.log_softmax(self.linear(hidden).view(-1, self.goal_area_dim), dim = 1)

model_LecTrackGoalAreaClassifier = LecTrackGoalAreaClassifier(hidden_dim = HIDDEN_DIM,
                                                             goal_area_dim = GOAL_AREA_DIM)

model_LecTrackGoalAreaClassifier = model_LecTrackGoalAreaClassifier.to(DEVICE)

optimizer_LecTrackGoalAreaClassifier = optim.Adam(model_LecTrackGoalAreaClassifier.parameters(), lr = 1e-4)

In [13]:
def get_index_and_score(turn, token_to_index, device):
    indexes = []
    scores = []
    token_score_list = turn["system"] + turn["user"]
    for token, score in token_score_list:
        if token not in token_to_index:
            indexes.append(token_to_index["<unk>"])
        else:
            indexes.append(token_to_index[token])
        scores.append(score)
    assert len(indexes) == len(scores)
    return torch.tensor(indexes, dtype = torch.long, device = device), torch.tensor(scores, dtype = torch.float, device = device)

In [14]:
def retrieve_gold_Method(raw_Y, ontology, device):
    ontology_methods = ontology["method"]
    raw_goal_method = raw_Y["method"]
    goal_method = ontology_methods.index(raw_goal_method)
    return torch.tensor([goal_method], dtype = torch.long, device = device)

def retrieve_gold_Requested(raw_Y, ontology, device):
    ontology_requestable = ontology["requestable"]
    raw_gold_requested = raw_Y["requested"]
    gold_requested = np.zeros(int(math.pow(2, len(ontology_requestable))), dtype = int)
    
    binary_representation = np.zeros(len(ontology_requestable), dtype = int)
    if len(raw_gold_requested) != 0:
        for requested in raw_gold_requested:
            binary_representation[ontology_requestable.index(requested)] = 1        
    
    goal_requested = binary_representation.dot(2**np.arange(binary_representation.size)[::-1])
    return torch.tensor([goal_requested], dtype = torch.long, device = device)

def retrieve_gold_GoalFood(raw_Y, ontology, device):
    ontology_informable_food = ontology["informable"]["food"]
    raw_goal_food = raw_Y["goal"]["food"]
    goal_food = 0
    if raw_goal_food != None:
        if raw_goal_food == "dontcare":
            goal_food = 1
        else:    
            goal_food = ontology_informable_food.index(raw_goal_food) + 2
    return torch.tensor([goal_food], dtype = torch.long, device = device)

def retrieve_gold_GoalPriceRange(raw_Y, ontology, device):
    ontology_informable_pricerange = ontology["informable"]["pricerange"]
    raw_goal_pricerange = raw_Y["goal"]["pricerange"]
    goal_pricerange = 0
    if raw_goal_pricerange != None:
        if raw_goal_pricerange == "dontcare":
            goal_pricerange = 1
        else:    
            goal_pricerange = ontology_informable_pricerange.index(raw_goal_pricerange) + 2
    return torch.tensor([goal_pricerange], dtype = torch.long, device = device)

def retrieve_gold_GoalName(raw_Y, ontology, device):
    ontology_informable_name = ontology["informable"]["name"]
    raw_goal_name = raw_Y["goal"]["name"]
    goal_name = 0
    if raw_goal_name != None:
        if raw_goal_name == "dontcare":
            goal_name = 1
        else:    
            goal_name = ontology_informable_name.index(raw_goal_name) + 2
    return torch.tensor([goal_name], dtype = torch.long, device = device)

def retrieve_gold_GoalArea(raw_Y, ontology, device):
    ontology_informable_area = ontology["informable"]["area"]
    raw_goal_area = raw_Y["goal"]["area"]
    goal_area = 0
    if raw_goal_area != None:
        if raw_goal_area == "dontcare":
            goal_area = 1
        else:    
            goal_area = ontology_informable_area.index(raw_goal_area) + 2
    return torch.tensor([goal_area], dtype = torch.long, device = device)

In [ ]:
# define the loss function
loss_function = nn.CrossEntropyLoss()

NUM_EPOCHS = 100

for epoch in range(NUM_EPOCHS):
    logging.info("Epoch\t{}/{}".format(epoch, NUM_EPOCHS))
    epoch_loss = 0
    # for each dialog
    for raw_X_train_dialog, raw_Y_train_dialog in tqdm(zip(raw_X_train, raw_Y_train), total = len(raw_X_train)):

        model_LecTrackEncoder.hidden = model_LecTrackEncoder.init_hidden()
        model_LecTrackEncoder.zero_grad()
        model_LecTrackMethodClassifier.zero_grad()
        model_LecTrackRequestedClassifier.zero_grad()
        model_LecTrackGoalFoodClassifier.zero_grad()
        model_LecTrackGoalPricerangeClassifier.zero_grad()
        model_LecTrackGoalNameClassifier.zero_grad()
        model_LecTrackGoalAreaClassifier.zero_grad()

        # for each turn in the dialog
        for raw_X_train_turn, raw_Y_train_turn in zip(raw_X_train_dialog, raw_Y_train_dialog):

            indexes, scores = get_index_and_score(raw_X_train_turn, token_to_index, device = DEVICE)

            output_LecTrackEncoder = model_LecTrackEncoder(indexes, scores)
            output_LecTrackMethodClassifier = model_LecTrackMethodClassifier(output_LecTrackEncoder[0])
            output_LecTrackRequestedClassifier = model_LecTrackRequestedClassifier(output_LecTrackEncoder[0])
            output_LecTrackGoalFoodClassifier = model_LecTrackGoalFoodClassifier(output_LecTrackEncoder[0])
            output_LecTrackGoalPricerangeClassifier = model_LecTrackGoalPricerangeClassifier(output_LecTrackEncoder[0])
            output_LecTrackGoalNameClassifier = model_LecTrackGoalNameClassifier(output_LecTrackEncoder[0])
            output_LecTrackGoalAreaClassifier = model_LecTrackGoalAreaClassifier(output_LecTrackEncoder[0])

            gold_LecTrackMethodClassifier = retrieve_gold_Method(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackRequestedClassifier = retrieve_gold_Requested(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackGoalFoodClassifier = retrieve_gold_GoalFood(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackGoalPricerangeClassifier = retrieve_gold_GoalPriceRange(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackGoalNameClassifier = retrieve_gold_GoalName(raw_Y_train_turn, ontology, device = DEVICE)
            gold_LecTrackGoalAreaClassifier = retrieve_gold_GoalArea(raw_Y_train_turn, ontology, device = DEVICE)

            loss_LecTrackMethodClassifier = loss_function(output_LecTrackMethodClassifier, gold_LecTrackMethodClassifier)
            loss_LecTrackRequestedClassifier = loss_function(output_LecTrackRequestedClassifier, gold_LecTrackRequestedClassifier)
            loss_LecTrackGoalFoodClassifier = loss_function(output_LecTrackGoalFoodClassifier, gold_LecTrackGoalFoodClassifier)
            loss_LecTrackGoalPricerangeClassifier = loss_function(output_LecTrackGoalPricerangeClassifier, gold_LecTrackGoalPricerangeClassifier)
            loss_LecTrackGoalNameClassifier = loss_function(output_LecTrackGoalNameClassifier, gold_LecTrackGoalNameClassifier)
            loss_LecTrackGoalAreaClassifier = loss_function(output_LecTrackGoalAreaClassifier, gold_LecTrackGoalAreaClassifier)

            loss = loss_LecTrackMethodClassifier + \
                    loss_LecTrackRequestedClassifier + \
                    loss_LecTrackGoalFoodClassifier + \
                    loss_LecTrackGoalPricerangeClassifier + \
                    loss_LecTrackGoalNameClassifier + \
                    loss_LecTrackGoalAreaClassifier
            loss.backward(retain_graph = True)
            
            epoch_loss += loss

            optimizer_LecTrackEncoder.step()
            optimizer_LecTrackMethodClassifier.step()
            optimizer_LecTrackRequestedClassifier.step()
            optimizer_LecTrackGoalFoodClassifier.step()
            optimizer_LecTrackGoalPricerangeClassifier.step()
            optimizer_LecTrackGoalNameClassifier.step()
            optimizer_LecTrackGoalAreaClassifier.step()
    logging.info("Epoch loss\t{}".format(epoch_loss))

INFO:root:Epoch	0/100
100%|██████████| 3224/3224 [02:26<00:00, 26.12it/s]
INFO:root:Epoch loss	129064.8203125
INFO:root:Epoch	1/100
100%|██████████| 3224/3224 [02:26<00:00, 27.34it/s]
INFO:root:Epoch loss	65721.9921875
INFO:root:Epoch	2/100
 31%|███       | 997/3224 [00:45<01:16, 29.27it/s]